In [1]:
import numpy as np
from numba import njit
from sklearn.linear_model import Lasso

In [2]:
@njit
def soft_threshold_numba(rho, lamda, w):
    if rho < -lamda * w:
        return rho + lamda * w
    elif rho > lamda * w:
        return rho - lamda * w
    else:
        return 0.0

In [3]:
@njit
def get_lamda_path_numba(X, y):
    epsilon = 0.0001
    K = 100
    m, p = X.shape

    y = y.reshape((m, 1))
    sx = X
    sy = y

    lambda_max = np.max(np.abs(np.sum(sx * sy, axis=0))) / m
    lamda_path = np.exp(
        np.linspace(np.log(lambda_max), np.log(lambda_max * epsilon), np.int64(K))
    )

    return lamda_path

In [4]:
@njit
def count_non_zero_coeffs(theta_vec):
    s = 0
    for i in theta_vec:
        if np.abs(i) > 1e-04:
            s += 1
    return s

In [54]:
@njit
def lasso_numba(
    X,
    y,
    lamda_path=None,
    penalty_factors=None,
    theta=None,
    num_iters=100,
    intercept=True,
    thresh=1e-7,
    active_thresh=1e-7,
    warm_start=True,
):

    m, p = X.shape

    x_mean = np.zeros((p,), dtype=np.float64)

    for i in range(p):
        x_mean[i] = X[:, i].mean()

    x_std = np.zeros((p,), dtype=np.float64)

    for i in range(p):
        x_std[i] = X[:, i].std()

    y_mean = np.mean(y)
    y_std = np.std(y)

    X_standardized = (X - x_mean) / x_std
    y_standardized = (y - y_mean) / y_std

    if intercept:
        X_tmp = np.ones((m, p + 1))
        X_tmp[:, 1:] = X
        X = X_tmp
    
    if lamda_path is None:
        path = m * get_lamda_path_numba(X=X_standardized, y=y_standardized)
    else: 
        path = m * lamda_path

    if intercept:
        X_tmp = np.ones((m, p + 1))
        X_tmp[:, 1:] = X_standardized
        X_standardized = X_tmp

    m, p = X_standardized.shape

    if theta is None:
        theta = np.zeros((p, 1))

    if penalty_factors is None:
        penalty_factors = np.ones((p, 1))

    lamdas = []
    thetas = []
    thetas_nat = []
    BIC = []
    
    for lamda in path:
        if not warm_start:
            theta = np.zeros((p, 1))
        sec_check_all_converged = False
        active_set = np.arange(p)
        active_set_converged = False

        for _i in range(num_iters):
            if (active_set.size != 0) and (not active_set_converged):
                active_set_converged_check = np.full((len(active_set),), False)
                active_set_update = np.full((len(active_set),), True)

                for subindex, j in enumerate(active_set):
                    w_j = penalty_factors[j].item()

                    y_pred = X_standardized @ theta

                    rho = 0.0
                    z = 0.0

                    for obs in range(m):
                        rho += X_standardized[obs, j].item() * (
                            y_standardized[obs].item()
                            - y_pred[obs].item()
                            + theta[j].item() * X_standardized[obs, j].item()
                        )
                        z += np.square(X_standardized[obs, j].item())

                    if intercept:
                        if j == 0:
                            tmp = rho / z
                            if np.abs(tmp) < active_thresh:
                                active_set_update[subindex] = False
                            if np.abs(theta[j] - tmp) < thresh:
                                active_set_converged_check[subindex] = True
                            theta[j] = tmp
                        else:
                            tmp = (1 / z) * soft_threshold_numba(rho, lamda, w_j)
                            if np.abs(tmp) < active_thresh:
                                active_set_update[subindex] = False
                            if np.abs(theta[j] - tmp) < thresh:
                                active_set_converged_check[subindex] = True
                            theta[j] = tmp

                    else:
                        tmp = (1 / z) * soft_threshold_numba(rho, lamda, w_j)
                        if np.abs(tmp) < active_thresh:
                            active_set_update[subindex] = False
                        if np.abs(theta[j] - tmp) < thresh:
                            active_set_converged_check[subindex] = True
                        theta[j] = tmp

                active_set_converged = np.all(active_set_converged_check)
                active_set = active_set[active_set_update]

            elif not sec_check_all_converged:
                active_set = np.arange(p)

                active_set_converged_check = np.full((len(active_set),), False)
                active_set_update = np.full((len(active_set),), True)

                m, p = X_standardized.shape

                for subindex, j in enumerate(active_set):
                    w_j = penalty_factors[j].item()

                    y_pred = X_standardized @ theta
                    rho = 0.0
                    z = 0.0

                    for obs in range(m):
                        rho += X_standardized[obs, j].item() * (
                            y_standardized[obs].item()
                            - y_pred[obs].item()
                            + theta[j].item() * X_standardized[obs, j].item()
                        )
                        z += np.square(X_standardized[obs, j].item())

                    if intercept:
                        if j == 0:
                            tmp = rho / z
                            if np.abs(tmp) < active_thresh:
                                active_set_update[subindex] = False
                            if np.abs(theta[j] - tmp) < thresh:
                                active_set_converged_check[subindex] = True
                            theta[j] = tmp
                        else:
                            tmp = (1 / z) * soft_threshold_numba(rho, lamda, w_j)
                            if np.abs(tmp) < active_thresh:
                                active_set_update[subindex] = False
                            if np.abs(theta[j] - tmp) < thresh:
                                active_set_converged_check[subindex] = True
                            theta[j] = tmp

                    else:
                        tmp = (1 / z) * soft_threshold_numba(rho, lamda, w_j)
                        if np.abs(tmp) < active_thresh:
                            active_set_update[subindex] = False
                        if np.abs(theta[j] - tmp) < thresh:
                            active_set_converged_check[subindex] = True
                        theta[j] = tmp

                active_set_converged = np.all(active_set_converged_check)
                active_set = active_set[active_set_update]

                if active_set_converged:
                    sec_check_all_converged = True
                    break
            else:
                break
        
        if not intercept:
            theta_tmp = theta.flatten() / x_std * y_std
        if intercept:
            theta_0 = (
                theta.flatten()[0] - np.sum((x_mean / x_std) * theta.flatten()[1:])
            ) * y_std + y_mean
            theta_betas = theta.flatten()[1:] / x_std * y_std
            theta_tmp = np.ones((p,))
            theta_tmp[1:] = theta_betas
            theta_tmp[0] = theta_0
            
        m, p = X.shape
        theta_bic = np.ones((p, 1))
        theta_bic[:,0] = theta_tmp
        residuals_hat = np.sum(np.square(y - X @ theta_bic))
        df_lamda = count_non_zero_coeffs(theta_vec=theta_bic.flatten())
        BIC_lasso = residuals_hat / (m * y_std**2) + np.log(m)/ m * df_lamda

        lamdas.append(lamda / m)
        thetas.append(np.copy(theta))
        thetas_nat.append(theta_tmp)
        BIC.append(BIC_lasso)

    return lamdas, thetas, thetas_nat, BIC

In [6]:
np.random.seed(seed=1)
n = 3000
X = np.random.rand(n,15)
y = np.array(13* X[:,7] + X[:,3] + X[:,5] + X[:,6] +X[:,7] +X[:,8]- 1.5 * X[:,0] - 14.5 * X[:,1] + 5 + np.random.normal(0,0.5,n), dtype=np.float64).reshape(-1,1)

In [7]:
# np.savetxt('test.csv', data, delimiter=',') 

In [39]:
def sk_learn_lasso(X, y, intercept=True, lamda_path=None):
    
    m, p = X.shape
    
    x_mean = X.mean(axis=0)
    x_std = X.std(axis=0)

    y_mean = np.mean(y)
    y_std = np.std(y)

    X_std = (X - x_mean) / x_std
    y_std = (y - y_mean) / y_std

    if lamda_path is None:
        path = get_lamda_path_numba(X=X_std, y=y_std)
    else: 
        path = lamda_path

    y_std = y_std.flatten()

    lamdas = []
    coeffs = []
    
    for lamda in path:
        reg = Lasso(alpha= lamda, fit_intercept = intercept)
        reg.fit(X_std, y_std)
        
        if intercept:
            coef = np.insert(arr=reg.coef_, obj=0, values=reg.intercept_)
        else:
            coef = reg.coef_
        
        lamdas.append(lamda)
        coeffs.append(np.copy(coef))

    return lamdas, coeffs

In [10]:
def soft_threshold(rho, lamda, w):
    """Soft threshold function used for normalized data and lasso regression"""
    if rho < -lamda * w:
        return rho + lamda * w
    elif rho > lamda * w:
        return rho - lamda * w
    else:
        return 0

In [34]:
res3 = naive_lasso(
    X = X,
    y = y,
    lamda_path= np.array([0.01]),
    intercept=False)

In [40]:
res = sk_learn_lasso(X = X, y = y, lamda_path = np.array([0.01]), intercept = False)

In [67]:
result_intercept = lasso_numba(X = X,
                         y = y,
                         lamda_path = np.array([0.01]),
                         intercept = True)
    
expected_result_intercept = sk_learn_lasso(X = X, y = y, intercept=True, lamda_path=np.array([0.01]))

result_no_intercept = lasso_numba(X = X,
                     y = y,
                     lamda_path = np.array([0.01]),
                     intercept = False)

expected_result_no_intercept = sk_learn_lasso(X = X, y = y, intercept=False, lamda_path=np.array([0.01]))

np.testing.assert_array_almost_equal(result_intercept[1][0].flatten(), expected_result_intercept[1][0], decimal=6)
np.testing.assert_array_almost_equal(result_no_intercept[1][0].flatten(), expected_result_no_intercept[1][0], decimal=6)

In [46]:
result_intercept

[{'lamda': 0.01,
  'theta_std': array([-2.41053011e-15, -6.25255210e-02, -6.90906757e-01,  0.00000000e+00,
          3.64002974e-02,  0.00000000e+00,  3.29210595e-02,  3.68611692e-02,
          6.66545167e-01,  3.85717584e-02,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00]),
  'theta_nat': array([  5.39898735,  -1.30382941, -14.27185171,   0.        ,
           0.76022267,   0.        ,   0.68572247,   0.76053048,
          13.79248918,   0.79279822,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ])}]

In [65]:
res2 = lasso_numba(X = X,
            y = y,
            lamda_path = np.array([0.01]),
            intercept = True)

In [66]:
res2[1][0].flatten()

array([-2.84849372e-15, -6.25255208e-02, -6.90906757e-01,  0.00000000e+00,
        3.64002973e-02,  0.00000000e+00,  3.29210594e-02,  3.68611692e-02,
        6.66545167e-01,  3.85717584e-02,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [32]:
np.delete(res2[1][index_lamda_opt], 0).reshape((15,1))

array([[-0.0706267 ],
       [-0.69786882],
       [ 0.        ],
       [ 0.04377336],
       [ 0.        ],
       [ 0.04102069],
       [ 0.04497576],
       [ 0.67379436],
       [ 0.04647234],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

In [93]:
%%time
res = lasso_numba(X = X, y = y, lamda_path = np.array([100, 80, 40,20 ,10,6, 5, 3, 2, 1, 0.5 , 0.01]))

CPU times: user 1.64 s, sys: 280 ms, total: 1.92 s
Wall time: 1.58 s


In [45]:
def adaptive_lasso(X, y, intercept=True, lamda_path= None, gamma_path = None , first_stage = "Lasso", num_iters = 100):
    
    m, p = X.shape
    
    if gamma_path is None:
        path_gamma = np.array([0.001, 0.01, 0.1, 0.5, 1, 2, 3, 4, 6, 8])
    else:
        path_gamma = gamma_path
        
    if first_stage == "OLS":
        reg = LinearRegression(fit_intercept=intercept).fit(X, y)
        coeffs = reg.coef_.T
    elif first_stage == "Lasso":
        res = lasso_numba(X = X, y = y)
        
        index_lamda_opt = np.where(res[3] == np.amin(res[3]))[0][0]
        coeffs = np.delete(res[1][index_lamda_opt], 0).reshape((p,1))
        
    else:
        raise AssertionError("This feature has so far only been implemented for OLS as its first-stage estimator.")
    
    coeffs[np.abs(coeffs) < 1.00e-15] = 1.00e-15
    
    results = []
    for gamma in path_gamma:
        
        if intercept:
            weights = np.ones((p + 1, 1))
            weights[1:, :] = 1.0 / np.abs(coeffs)**gamma
        else:
            weights = 1.0 / np.abs(coeffs)**gamma
        
        res = lasso_numba(X,
                    y,
                    lamda_path=lamda_path,
                    penalty_factors=weights,
                    theta=None,
                    num_iters=num_iters,
                    intercept=intercept,
                    thresh=1e-7,
                    active_thresh=1e-7,
                    warm_start=True)
        
        results.append(res)
    
    return path_gamma, results

In [22]:
from sklearn.linear_model import LinearRegression

In [54]:
%%time
real = adaptive_lasso(X = X, y= y, gamma_path = np.array([1]), lamda_path = np.array([0.0001]), first_stage = "Lasso")
#real = adaptive_lasso(X = X, y= y)

CPU times: user 1.49 s, sys: 4.3 s, total: 5.78 s
Wall time: 217 ms


In [55]:
real

(array([1]),
 [([9.999999999999999e-05], [array([[-2.92377234e-15],
           [-7.17216222e-02],
           [-7.00123379e-01],
           [ 0.00000000e+00],
           [ 4.38723755e-02],
           [ 0.00000000e+00],
           [ 4.11435243e-02],
           [ 4.51878148e-02],
           [ 6.76071968e-01],
           [ 4.68359152e-02],
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00],
           [ 0.00000000e+00]])], [array([  5.15611627,  -1.49559346, -14.46223667,   0.        ,
             0.91627753,   0.        ,   0.85699062,   0.93232828,
            13.98962256,   0.96265847,   0.        ,   0.        ,
             0.        ,   0.        ,   0.        ,   0.        ])], [0.028341263226400813])])